In [1]:
# Geopandas，其他GIS工具Pyecharts
# 依赖包需要依次安装，最后安装Geopandas：
# GDAL、Fina、pyproj、Rtree、shapely、
# GDAL的安装：①brew install GDAL安装GDAL;②brew info GDAL查看GDAL版本；③pip install GDAL==3.5.2（与brew安装的版本一致）
# 安装完的GDAL无法直接import gdal，但是from osgeo import gdal却可以
# 因此，写以下脚本放置与site-packages文件夹中
# from osgeo.gdal import deprecation_warn
# deprecation_warn('gdal')
# from osgeo.gdal import *
# pycharm无法使用gdal和geopandas，因此使用jupyter notebook进行绘图。

In [12]:
import geopandas as gpd

print('OK!')

OK!


In [14]:
import shapely.geometry as geo
pnt = geo.Point(0,0)
print('This distance between point(0.0) and (1,1) is:', pnt.distance(geo.Point(1,1)))
gpd.GeoSeries([geo.Point(0,0),
               geo.Point(0,1),
               geo.Point(1,1),
               geo.Point(1,0)],
             index=['a','b','c','d'])

This distance between point(0.0) and (1,1) is: 1.4142135623730951


a    POINT (0.00000 0.00000)
b    POINT (0.00000 1.00000)
c    POINT (1.00000 1.00000)
d    POINT (1.00000 0.00000)
dtype: geometry